In [14]:
import pandas as pd
from statsmodels.tsa.api import ExponentialSmoothing

In [15]:
dataset = pd.read_excel('Training_Dataset_Harsh.xlsx', sheet_name = 'Sheet1')
dataset['Date'] = pd.to_datetime(dataset['Date'])
inflows = dataset[dataset['Type']=='Inflow']
monthly = inflows.groupby(inflows['Date'].dt.to_period('M'))['Amount'].sum()
monthly.index = monthly.index.to_timestamp()

In [16]:
model = ExponentialSmoothing(monthly, trend='add').fit()

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [17]:
forecast = model.forecast(6)

In [18]:
forecast_dates = pd.date_range(start=inflows['Date'].max() + pd.offsets.MonthBegin(1), periods=6, freq='MS')
forecast_df = pd.DataFrame({'Date': forecast_dates, 'Forecast': forecast.values})
forecast_df.to_excel('forecast.xlsx', index=False)

In [19]:
#result = pd.DataFrame({'Date': forecast.index, 'Forecast': forecast.values})
#result.to_excel('Forecast.xlsx', index=False)

In [25]:
outflows = dataset[dataset['Type']=='Outflow']
opex_categories = ['Salaries', 'Rent', 'Marketing', 'Software']
estimated_opex = outflows[outflows['Category'].isin(opex_categories)]
monthly_opex = estimated_opex.groupby(estimated_opex['Date'].dt.to_period('M'))['Amount'].sum().mean()
annual_opex = monthly_opex * 12
annual_capex = 500

In [26]:
avg_monthly_forecast = forecast_df['Forecast'].mean()
annual_revenue = avg_monthly_forecast * 12
fcf_year1 = annual_revenue - annual_opex - annual_capex

In [27]:
growth_rate = 0.08
fcfs = [fcf_year1]
for i in range(1,5):
  fcfs.append(fcfs[-1] * (1+growth_rate))

In [28]:
years = pd.date_range(start=forecast_dates[0], periods=5, freq='YS').year
fcf_df = pd.DataFrame({'Year': years, 'FCF': fcfs})

In [29]:
fcf_df.to_excel('fcf_forecast.xlsx',index=False)